## Processamento dos dados

In [99]:
import pandas as pd
import numpy as np

### Criar uma tabela com todos os MQLs, com todas as informações disponíveis pra cada um:

In [100]:
# Importando os dados:
df_deals = pd.read_table('olist_closed_deals_dataset.csv', sep=',', decimal='.', parse_dates=['won_date'])
df_mql = pd.read_table('olist_marketing_qualified_leads_dataset.csv', sep=',', parse_dates=['first_contact_date'])

In [101]:
# Juntando as duas bases de dados através da coluna "mql_id":
df = df_mql.merge(df_deals, how='left', on='mql_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   mql_id                         8000 non-null   object        
 1   first_contact_date             8000 non-null   datetime64[ns]
 2   landing_page_id                8000 non-null   object        
 3   origin                         7940 non-null   object        
 4   seller_id                      842 non-null    object        
 5   sdr_id                         842 non-null    object        
 6   sr_id                          842 non-null    object        
 7   won_date                       842 non-null    datetime64[ns]
 8   business_segment               841 non-null    object        
 9   lead_type                      836 non-null    object        
 10  lead_behaviour_profile         665 non-null    object        
 11  has_company      

### Criar uma coluna com quais MQLs fecharam acordo e quais foram perdidos:

In [102]:
df['deal'] = np.where(np.isnat(df['won_date']), 0, 1)

In [103]:
df.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,deal
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0,1


## Perguntas a serem respondidas

1. Qual foi a taxa de conversão total?
2. Qual foi a taxa de conversão de cada origem?
3. Qual foi a taxa de conversão de cada página inicial?
4. Para cada SR, qual a receita média declarada dos leads?
5. Para cada SR, quantos de cada business_type a pessoa converteu?
6. Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?
7. Para cada SR, quantos de cada lead_type a pessoa converteu?

Extra:

Caso consiga resolver as questões acima com facilidade, aproveite a base "irmã" dessa para fazer outras explorações que você ache interessante!

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

Lembre de separar novamente em 2 etapas: 
1. Processamento dos dados, para criar uma tabela "analítica".
2. Análise exploratória dos dados.

### 1. Qual foi a taxa de conversão total?

In [105]:
total_convertion_rate = df['deal'].sum()

In [106]:
total_convertion_rate = df['deal'].sum() / df['deal'].shape[0]
total_convertion_rate = total_convertion_rate * 100
total_convertion_rate = round(total_convertion_rate, 2)
print(f'A taxa de conversão total foi de {total_convertion_rate}%.')

A taxa de conversão total foi de 10.52%.


### 2. Qual foi a taxa de conversão de cada origem?

In [107]:
# Substituindo o np.nan da coluna "origin":
df['origin'] = df['origin'].fillna('n/a')

In [108]:
df['origin'].unique().tolist()

['social',
 'paid_search',
 'organic_search',
 'email',
 'unknown',
 'referral',
 'direct_traffic',
 'display',
 'n/a',
 'other_publicities',
 'other']

In [109]:
closed_deals = df[df['deal'] == 1]
not_closed_deals = df[df['deal'] == 0]

In [110]:
closed_deals_sum = closed_deals.groupby(['origin']).agg({'deal': 'sum'}).reset_index()

not_closed_deals_sum = not_closed_deals.groupby(['origin']).agg({'deal': 'count'}).reset_index()

In [111]:
convertion_rate_by_origin = pd.merge(closed_deals_sum, not_closed_deals_sum, on=['origin'])

In [112]:
convertion_rate_by_origin.columns = ['origin', 'deal_closed', 'deal_not_closed']

In [113]:
convertion_rate_by_origin['conversion_rate'] = (
                            (convertion_rate_by_origin['deal_closed'] / 
                            (convertion_rate_by_origin['deal_closed'] + convertion_rate_by_origin['deal_not_closed']))
                            * 100).round(2)

In [114]:
print('O DataFrame a seguir mostra a taxa de conversão dos acordos por origem:')
convertion_rate_by_origin.drop(columns=['deal_closed', 'deal_not_closed'])

O DataFrame a seguir mostra a taxa de conversão dos acordos por origem:


,origin,conversion_rate
0,direct_traffic,11.22
1,display,5.08
2,email,3.04
3,n/a,23.33
4,organic_search,11.80
5,other,2.67
6,other_publicities,4.62
7,paid_search,12.30
8,referral,8.45
9,social,5.56


### 3. Qual foi a taxa de conversão de cada página inicial?

In [115]:
df_page_conversion = df.groupby('landing_page_id')['deal'].agg(['sum','count'])
df_page_conversion['%_page_conversao'] = df_page_conversion['sum'] / df_page_conversion['count']
df_page_conversion = df_page_conversion[df_page_conversion['%_page_conversao'] > 0]
df_page_conversion['%_page_conversao'] = df_page_conversion['%_page_conversao'] * 100
df_page_conversion['%_page_conversao'] = round(df_page_conversion['%_page_conversao'], 2)
df_page_conversion = df_page_conversion[['%_page_conversao']]

In [116]:
print('O DataFrame a seguir mostra a taxa de conversão de cada página inicial:')
df_page_conversion

O DataFrame a seguir mostra a taxa de conversão de cada página inicial:


,%_page_conversao
landing_page_id,
007f9098284a86ee80ddeb25d53e0af8,6.06
0218f6be0b76aca72ab4d00ee9e8cf10,12.50
04a75b75d5f4f3a16b0178fbe382da00,100.00
0625ca637bf063abf254a66fe91fa84b,33.33
090ffa1062e6dde353c36524563acefc,33.33
...,...
fa5a0151e1f37d0055a3478fbb48827a,26.67
fa6f59a21825d841e307151460fb0302,25.00
fbc24da54d531c6204ae2d17b1090bb1,20.83


### 4. Para cada SR, qual a receita média declarada dos leads?

In [117]:
declared_monthly_revenue_by_sr = pd.DataFrame(
    df.groupby('sr_id')['declared_monthly_revenue'].mean().round(2).sort_values(ascending = False)).reset_index()


print('O DataFrame a seguir mostra a receita mensal média declarada por cada um dos {} SRs:'.format(
    declared_monthly_revenue_by_sr.shape[0]))

declared_monthly_revenue_by_sr

O DataFrame a seguir mostra a receita mensal média declarada por cada um dos 22 SRs:


,sr_id,declared_monthly_revenue
0,4ef15afb4b2723d8f3d81e51ec7afefe,377293.23
1,de63de0d10a6012430098db33c679b0b,151415.09
2,9749123c950bf8363ace42cb1c2d0815,103571.43
3,9d12ef1a7eca3ec58c545c678af7869c,43333.33
4,4b339f9567d060bcea4f5136b9f5949e,27777.78
5,a8387c01a09e99ce014107505b92388c,24038.69
6,34d40cdaf94010a1d05b0d6212f9e909,19000.00
7,56bf83c4bb35763a51c2baab501b4c67,12500.00
8,6aa3b86a83d784b05f0e37e26b20860d,8000.00
9,068066e24f0c643eb1d089c7dd20cd73,7037.04


### 5. Para cada SR, quantos de cada business_type a pessoa converteu?

In [118]:
df_sr_business_type = df.groupby(['sr_id', 'business_type'])['deal'].agg(['sum'])

In [119]:
print('O DataFrame a seguir mostra quantos de cada business_type cada SR converteu:')
df_sr_business_type

O DataFrame a seguir mostra quantos de cada business_type cada SR converteu:


sum
sr_id                            business_type     
060c0a26f19f4d66b42e0d8796688490 manufacturer     9
                                 reseller        23
068066e24f0c643eb1d089c7dd20cd73 manufacturer     6
                                 reseller        21
0a0fb2b07d841f84fb6714e35c723075 other            1
2695de1affa7750089c0455f8ce27021 manufacturer    16
                                 reseller        42
34d40cdaf94010a1d05b0d6212f9e909 manufacturer     1
                                 other            1
                                 reseller         8
495d4e95a8cf8bbf8b432b612a2aa328 manufacturer    14
                                 reseller        47
4b339f9567d060bcea4f5136b9f5949e manufacturer     2
                                 reseller         6
4ef15afb4b2723d8f3d81e51ec7afefe manufacturer    43
                                 reseller        90
56bf83c4bb35763a51c2baab501b4c67 manufacturer     8
                                 reseller        16
6565aa9ce3178a5caf6171827af3a9ba manufacturer    19
                                 reseller        54
6aa3b86a83d784b05f0e37e26b20860d reseller         1
85fc447d336637ba1df43e793199fbc8 manufacturer    25
                                 reseller        39
9749123c950bf8363ace42cb1c2d0815 manufacturer     2
                                 reseller         5
9ae085775a198122c5586fa830ff7f2b manufacturer    15
                                 reseller        36
9d12ef1a7eca3ec58c545c678af7869c manufacturer     2
                                 reseller         4
9e4d1098a3b0f5da39b0bc48f9876645 manufacturer    10
                                 reseller        14
a8387c01a09e99ce014107505b92388c manufacturer     4
                                 other            1
                                 reseller        21
b90f87164b5f8c2cfa5c8572834dbe3f reseller         1
c638112b43f1d1b86dcabb0da720c901 manufacturer    12
                                 reseller        22
d3d1e91a157ea7f90548eef82f1955e3 manufacturer    19
                                 reseller        61
de63de0d10a6012430098db33c679b0b manufacturer    12
                                 reseller        40
fbf4aef3f6915dc0c3c97d6812522f6a manufacturer    23
                                 reseller        36

### 6. Para cada SR, quantos de cada lead_behaviour_profile a pessoa converteu?

In [120]:
# Substituindo o np.nan da coluna "lead_behaviour_profile":
df['lead_behaviour_profile'] = df['lead_behaviour_profile'].fillna('n/a')

In [121]:
behaviour_profile_by_sr = df
behaviour_profile_by_sr['lead_behaviour_profile_split'] = behaviour_profile_by_sr.lead_behaviour_profile.str.split(', ')
behaviour_profile_by_sr = behaviour_profile_by_sr.explode('lead_behaviour_profile_split')

In [122]:
behaviour_profile_by_sr.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,deal,lead_behaviour_profile_split
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaT,NaN,NaN,n/a,NaN,NaN,NaN,NaN,NaN,NaN,0,n/a
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaT,NaN,NaN,n/a,NaN,NaN,NaN,NaN,NaN,NaN,0,n/a
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaT,NaN,NaN,n/a,NaN,NaN,NaN,NaN,NaN,NaN,0,n/a
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaT,NaN,NaN,n/a,NaN,NaN,NaN,NaN,NaN,NaN,0,n/a
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0,1,cat


In [123]:
print('O DataFrame a seguir mostra quantos lead_behaviour_profile de cada categoria foram convertidos por cada SR:')
pd.DataFrame(behaviour_profile_by_sr.groupby(['sr_id', 'lead_behaviour_profile_split'])['lead_behaviour_profile_split'].count())

O DataFrame a seguir mostra quantos lead_behaviour_profile de cada categoria foram convertidos por cada SR:


lead_behaviour_profile_split
sr_id                            lead_behaviour_profile_split                              
060c0a26f19f4d66b42e0d8796688490 cat                                                     15
                                 eagle                                                    2
                                 n/a                                                      8
                                 shark                                                    1
                                 wolf                                                     6
...                                                                                     ...
fbf4aef3f6915dc0c3c97d6812522f6a cat                                                     30
                                 eagle                                                    9
                                 n/a                                                     16
                                 shark                                                    2
                                 wolf                                                     2

[83 rows x 1 columns]

### 7. Para cada SR, quantos de cada lead_type a pessoa converteu?

In [124]:
lead_type_by_sr = df.groupby(['sr_id', 'lead_type'])['deal'].agg(['sum'])

In [125]:
print('O DataFrame a seguir mostra quantos de cada lead_type cada SR converteu:')
lead_type_by_sr

O DataFrame a seguir mostra quantos de cada lead_type cada SR converteu:


sum
sr_id                            lead_type           
060c0a26f19f4d66b42e0d8796688490 industry           8
                                 offline            1
                                 online_beginner    3
                                 online_big         2
                                 online_medium     15
...                                               ...
fbf4aef3f6915dc0c3c97d6812522f6a online_beginner    1
                                 online_big         8
                                 online_medium     30
                                 online_small       6
                                 online_top         2

[118 rows x 1 columns]